### BRIAN, THE STITCH, AND THE WARDROBE
Imagine that on the floor in front of you is a pile of all
the clothing you own—trousers, tops, shoes, and coats,
all of different styles. Your stylist, Brian, is becoming
increasingly frustrated with how long it takes him to
find the items you require, so he devises a clever plan.
He tells you to organize your clothes into a wardrobe
that is infinitely high and wide (Figure 3-1). When you
want to request a particular item, you simply need to
tell Brian its location and he will sew the item from
scratch using his trusty sewing machine. It soon
becomes obvious that you will need to place similar
items near to each other, so that Brian can accurately
re-create each item given only its location.

After several weeks of practice, you and Brian have
adjusted to each other’s understandings of the wardrobe
layout. It is now possible for you to tell Brian the location of any item of clothing that you desire, and he
can accurately sew it from scratch!
This gives you an idea—what would happen if you gave
Brian a wardrobe location that was empty? To your
amazement, you find that Brian is able to generate
entirely new items of clothing that haven’t existed
before! The process isn’t perfect, but you now have
limitless options for generating new clothing, just by
picking an empty location in the infinite wardrobe and
letting Brian work his magic with the sewing machine.


In [3]:
import numpy as np

In [2]:
from tensorflow.keras import datasets
(x_train,y_train), (x_test,y_test) = datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [5]:
def preprocess(imgs):
    imgs = imgs.astype('float32')/255.0
    imgs = np.pad(imgs, ((0,0),(2,2),(2,2)), constant_values=0.0)
    imgs = np.expand_dims(imgs,-1)
    return imgs


In [6]:
print(x_train.shape)
x_train = preprocess(x_train)
x_test = preprocess(x_test)
print(x_train.shape)

(60000, 28, 28)
(60000, 32, 32, 1)


An autoencoder is a neural network made up of two parts:
- An encoder network that compresses high-dimensional
input data such as an image into a lower-dimensional
embedding vector
- A decoder network that decompresses a given
embedding vector back to the original domain (e.g.,
back to an image)

#### Encoder

In [30]:
from tensorflow.keras import layers,models
import keras.backend as K 

In [33]:
encoder_input = layers.Input(shape=(32,32,1),name='encoder_input')
x = layers.Conv2D(filters=32
                  ,kernel_size=3
                  ,strides=2
                  ,activation='relu'
                  ,padding='same')(encoder_input)
x = layers.Conv2D(filters=64
                  ,kernel_size=3
                  ,strides=2
                  ,activation='relu'
                  ,padding='same')(x)
x = layers.Conv2D(filters=128
                  ,kernel_size=3
                  ,strides=2
                  ,activation='relu'
                  ,padding='same')(x)

shape_before_flattening = K.int_shape(x)[1:]

x = layers.Flatten()(x)
encoder_output = layers.Dense(units=2,name='encoder_output')(x)
model = models.Model(encoder_input,encoder_output)
model.summary()


Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 32, 32, 1)]       0         
                                                                 
 conv2d_47 (Conv2D)          (None, 16, 16, 32)        320       
                                                                 
 conv2d_48 (Conv2D)          (None, 8, 8, 64)          18496     
                                                                 
 conv2d_49 (Conv2D)          (None, 4, 4, 128)         73856     
                                                                 
 flatten_15 (Flatten)        (None, 2048)              0         
                                                                 
 encoder_output (Dense)      (None, 2)                 4098      
                                                                 
Total params: 96770 (378.01 KB)
Trainable params: 96770 (3

### Decoder

The convolutional transpose layer uses the same
principle as a standard convolutional layer (passing a
filter across the image), but is different in that setting
strides = 2 doubles the size of the input tensor in both
dimensions

In [26]:
decoder_input = layers.Input(shape=(2,), name='decoder_input')
x = layers.Dense(np.prod(shape_before_flattening))(decoder_input)
x = layers.Reshape(shape_before_flattening)(x)
x = layers.Conv2DTranspose(
    128,(3,3)
    ,strides=2
    ,activation='relu'
    ,padding='same'
)(x)

x = layers.Conv2DTranspose(
    64,(3,3)
    ,strides=2
    ,activation='relu'
    ,padding='same'
)(x)

x = layers.Conv2DTranspose(
    32,(3,3)
    ,strides=2
    ,activation='relu'
    ,padding='same'
)(x)

decoder_output = layers.Conv2D(
    1,(3,3)
    ,strides=1
    ,activation='sigmoid'
    ,padding='same'
    ,name='decoder_output'
)(x)
decoder = models.Model(decoder_input, decoder_output)

In [35]:
autoencoder = models.Model(encoder_input, decoder(encoder_output))